In [1]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
#using NearestNeighbors # Can't use KDtree in julia for posterior prediction

In [2]:
include("../../../util.j")

colnorm (generic function with 1 method)

In [3]:
# unnecessary packages #

#using Pkg
#Pkg.add("UnicodePlots")
using UnicodePlots   # check the structure of the sparse matrix
using BenchmarkTools

using StatsPlots
using MCMCChains
using PrettyTables

In [4]:
#using Pkg
#Pkg.add("NearestNeighborDescent");
#using NearestNeighborDescent

In [5]:
@load "../data/sim1data.jld";
@load "../results/ω_pos_mean_conj_NNGP.jld"
@load "../results/w_pos_mean_spbayes.jld"
@load "../results/ω_incp_obs_pos_qt_BSLMC.jld"

1-element Array{Symbol,1}:
 :ω_pos_BSLMC_mean

In [6]:
# plot the pictures of recovered latent process
using RCall
@rput ω_ord
@rput coords_ord
@rput S
@rput ω_pos_mean
@rput ω_pos_BSLMC_mean
@rput w_pos_mean_spbayes
R"""
library(MBA)
library(classInt)
library(RColorBrewer)
library(sp)
library(coda)
library(spBayes)
library(fields)

h <- 12
surf.raw1 <- mba.surf(cbind(t(coords_ord[, S]), ω_ord[S, 1]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.raw2 <- mba.surf(cbind(t(coords_ord[, S]), ω_ord[S, 2]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.fit1 <- mba.surf(cbind(t(coords_ord[, S]), ω_pos_mean[, 1]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.fit2 <- mba.surf(cbind(t(coords_ord[, S]), ω_pos_mean[, 2]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.BSLMC1 <- mba.surf(cbind(t(coords_ord[, S]), ω_pos_BSLMC_mean[, 1]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.BSLMC2 <- mba.surf(cbind(t(coords_ord[, S]), ω_pos_BSLMC_mean[, 2]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.bench1 <- mba.surf(cbind(t(coords_ord[, S]), w_pos_mean_spbayes[, 1]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.bench2 <- mba.surf(cbind(t(coords_ord[, S]), w_pos_mean_spbayes[, 2]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est

surf.brks <- classIntervals(c(surf.raw1[["z"]], surf.raw2[["z"]], surf.fit1[["z"]], surf.fit2[["z"]],
                surf.BSLMC1[["z"]], surf.BSLMC2[["z"]], surf.bench1[["z"]], surf.bench2[["z"]]), 500, 'pretty')$brks
col.pal <- colorRampPalette(brewer.pal(11,'RdBu')[11:1])
xlim <- c(0, 1.13)

zlim <- range(c(surf.raw1[["z"]], surf.raw2[["z"]], surf.fit1[["z"]], surf.fit2[["z"]]), 
                surf.BSLMC1[["z"]], surf.BSLMC2[["z"]], surf.bench1[["z"]], surf.bench2[["z"]])

# size for the mapping of w               
width <- 360
height <- 360
pointsize <- 16

png(paste("../../pics/sim1_map-w1-true_conj.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.raw1)
plot(t(coords_ord[, S]), typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim1_map-w2-true_conj.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.raw2)
plot(t(coords_ord[, S]), typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim1_map-w1-fit_conj.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.fit1)
plot(t(coords_ord[, S]), typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim1_map-w2-fit_conj.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.fit2)
plot(t(coords_ord[, S]), typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim1_map-w1-BSLMC.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.BSLMC1)
plot(t(coords_ord[, S]), typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim1_map-w2-BSLMC.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.BSLMC2)
plot(t(coords_ord[, S]), typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim1_map-w1-LMC_spbayes.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.bench1)
plot(t(coords_ord[, S]), typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim1_map-w2-LMC_spbayes.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.bench2)
plot(t(coords_ord[, S]), typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

"""

┌ Warning: Package RCall does not have AxisArrays in its dependencies:
│ - If you have RCall checked out for development and have
│   added AxisArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with RCall
│ Loading AxisArrays into RCall from project dependency, future warnings for RCall are suppressed.
└ @ nothing nothing:840
┌ Warning: RCall.jl: Loading required package: magic
│ Loading required package: abind
│ Loading required package: Formula
│ Loading required package: Matrix
└ @ RCall /home/lu/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Loading required package: spam
│ Loading required package: dotCall64
│ Loading required package: grid
│ Spam version 2.2-2 (2019-03-07) is loaded.
│ Type 'help( Spam)' or 'demo( spam)' for a short introduction 
│ and overview of this package.
│ Help for individual functions is also obtained by adding the
│ suffix '.spam' to 

RObject{IntSxp}
null device 
          1 
